In [1]:
import numpy as np
import pandas as pd
import os
from statsmodels.stats.multitest import fdrcorrection

In [2]:
# bim.data include all gwas snps info for pheno
df = pd.read_csv('data/data.bim', sep='\t', header=None)
df.columns = ['CHR', 'SNP', 'POS', 'BP', 'A1', 'A2']

In [3]:
pheno = 'bag'
df_pqtls_info = pd.read_csv(f'data/pqtls_interval_{pheno}.csv')

In [4]:
pqtls = df_pqtls_info['SOMAMER_ID'].tolist()

# gene position for calculate distance between pqtl and gene pos = floor((start_pos + end_pos) / 2)
gene_pos = np.floor((df_pqtls_info['start_b37'] + df_pqtls_info['end_b37']) / 2).to_list()
# chr for the gene
gene_chr = df_pqtls_info['chr_b37'].tolist()
# gene ensembl_id to identify the gene
gene_id = df_pqtls_info['ensembl_gene_id'].tolist()

# raw path for the pqtls
pqtl_path = 'interval_pqtl_data_raw'
# merged path for pqtls
merged_path = 'interval_pqtl_merged'

print(len(pqtls), 'pQTLs founded.')

24 pQTLs founded.


In [5]:
cis_pqtls_list = []
pqtls_list = []
for i, pqtl in enumerate(pqtls):
    print(i, ':', pqtl)
    if os.path.exists(os.path.join(pqtl_path, pqtl)):
        if os.path.exists(os.path.join(merged_path, f'{pqtl}.csv')):
            print(f'{pqtl} merged.')
            df_chr_all = pd.read_csv(os.path.join(merged_path, f'{pqtl}.csv'))
        else:
            # list gwas summary data for all chr
            files = os.listdir(os.path.join(pqtl_path, pqtl))
            # print(len(files))
            print(f'merging on {pqtl}')
            gwas_summary_chrs = []
            for f in files:
                if f.endswith('tsv') or f.endswith('tsv.gz'):
                    # read chr file
                    if f.endswith('gz'):
                        df_chr = pd.read_csv(os.path.join(pqtl_path, pqtl, f), sep='\t', compression='gzip')
                    else:
                        df_chr = pd.read_csv(os.path.join(pqtl_path, pqtl, f), sep='\t')
                    gwas_summary_chrs.append(df_chr)

            # concat all chr files
            df_chr_all = pd.concat(gwas_summary_chrs, axis=0)

            # rsid: merge with data.bims
            df_chr_all = pd.merge(df_chr_all, df[['SNP', 'BP', 'CHR']], left_on=['chromosome', 'position'], right_on=['CHR', 'BP'])

            # p-value
            df_chr_all['P'] = np.power(10, df_chr_all['log(P)'])

            # fdr corrected p-value
            p_fdr = fdrcorrection(df_chr_all['P'].to_numpy(), is_sorted=False)
            df_chr_all['FDR'] = p_fdr[1]

            # save merged result for future use
            df_chr_all.to_csv(os.path.join(merged_path, f'{pqtl}.csv'), index=False)

        print(f'{len(df_chr_all)} snps loaded for {pqtl}')
        # all snps that pass fdr correction
        df_sig = df_chr_all[df_chr_all['FDR'] < 0.05].copy()
        df_sig['ensembl_gene_id'] = gene_id[i]
        df_sig['gene_pos'] = gene_pos[i]
        # distance for the pqtls to the gene
        df_sig['distance'] = np.abs(df_sig['position'] - df_sig['gene_pos'])
        pqtls_list.append(df_sig)
        df_sig['id'] = pqtl
        # cis-pqtl in 250kb
        df_cis_pqtls = df_sig[df_sig['distance'] < 250 * 1000].copy()
        print(f'pqtls for {pqtl}: {len(df_sig)}, '
              f'cis-pqtls: {len(df_cis_pqtls)}')
        cis_pqtls_list.append(df_cis_pqtls)
        print('-' * 100)

0 : ABHD12.7825.7.3
ABHD12.7825.7.3 merged.
7255726 snps loaded for ABHD12.7825.7.3
pqtls for ABHD12.7825.7.3: 0, cis-pqtls: 0
----------------------------------------------------------------------------------------------------
1 : C1RL.9348.1.3
C1RL.9348.1.3 merged.
7255726 snps loaded for C1RL.9348.1.3
pqtls for C1RL.9348.1.3: 73, cis-pqtls: 71
----------------------------------------------------------------------------------------------------
2 : CA4.3177.49.2
CA4.3177.49.2 merged.
7255726 snps loaded for CA4.3177.49.2
pqtls for CA4.3177.49.2: 65, cis-pqtls: 0
----------------------------------------------------------------------------------------------------
3 : CD163.5028.59.1
CD163.5028.59.1 merged.
7255726 snps loaded for CD163.5028.59.1
pqtls for CD163.5028.59.1: 202, cis-pqtls: 21
----------------------------------------------------------------------------------------------------
4 : CDC25B.12427.8.3
CDC25B.12427.8.3 merged.
7255726 snps loaded for CDC25B.12427.8.3
pqtls for C

In [6]:
df_cis_pqtls_all = pd.concat(cis_pqtls_list, axis=0)
df_pqtl_all = pd.concat(pqtls_list, axis=0)

In [7]:
df_cis_pqtls_all.to_csv(os.path.join('pqtl_data', f'cis_pqtls_interval_{pheno}.csv'), index=False)
df_pqtl_all.to_csv(os.path.join('pqtl_data', f'pqtls_interval_{pheno}.csv'), index=False)

In [8]:
print(len(pd.unique(df_pqtl_all['ensembl_gene_id'])))

16
